# Brute force pulse sequence search

Using arbitrary units. Energy is normalized to the standard deviation in chemical shift strengths. Reduced Planck's constant $\hbar \equiv 1$.

## TODO

- [ ] Add rotation errors, phase errors
- [ ] Add ensemble measurement

In [6]:
import qutip as qt
import numpy as np
import sys

In [7]:
sys.path.append('..')

In [8]:
import pulse_sequences as ps

## Create system

In [56]:
delay = 1e-4  # time is relative to chemical shift strength
pulse_width = 2e-5
N = 3  # number of spins
ensemble_size = 5

In [38]:
X, Y, Z = ps.get_collective_spin(N=N)

In [57]:
Hsys_ensemble = [ps.get_Hsys(N=N, dipolar_strength=1) for _ in range(ensemble_size)]

In [58]:
pulses_ensemble = []
for H in Hsys_ensemble:
    r = np.random.normal(scale=.01)
    pt = np.random.normal(scale=1e-4)
    pulses_ensemble.append(
        ps.get_pulses(H, X, Y, Z, pulse_width=pulse_width, delay=delay,
                      rot_error=r, phase_transient=pt)
    )

In [59]:
Utarget = qt.tensor([qt.identity(2)]*N)

In [60]:
pulse_names = [
    'delay', 'X', 'Y', 'Xbar', 'Ybar'
]

## Define computational tree

Keep track of propagator in memory to reduce computation time.

In [23]:
class Node(object):
    
    def __init__(self, propagator, sequence=[], depth=0):
        self.propagator = propagator
        self.sequence = sequence
        self.depth = depth
        
        self.children = {}
    
    def has_children(self):
        return len(self.children) > 0
    
    def evaluate(self, Utarget, pulses, reward_dict, max_depth=6):
        """If the node isn't at max_depth, then create children and
        evaluate each individually. If the node is at max_depth, then
        calculate the reward and add the sequence/reward pair to
        reward_dict.
        
        Arguments:
            pulses: An array of unitary operators representing all actions
                that can be applied to the system.
        Returns: The maximum reward seen by the node or its children, and
            the corresponding sequence.
            
        """
        if self.depth < max_depth:
            max_reward = 0
            max_reward_sequence = []
            for i, pulse in enumerate(pulses):
                propagator = pulse * self.propagator
                child = Node(propagator,
                             self.sequence + [i],
                             depth=self.depth + 1)
                r, s = child.evaluate(Utarget, pulses, reward_dict, max_depth)
                if r > max_reward:
                    max_reward = r
                    max_reward_sequence = s
            return max_reward, max_reward_sequence
        else:
            fidelity = np.clip(
                qt.metrics.average_gate_fidelity(self.propagator, Utarget),
                0, 1
            )
            reward = - np.log10(1.0 - fidelity + 1e-100)
            sequence_str = ','.join([str(a) for a in self.sequence])
            reward_dict[sequence_str] = reward
            return reward, self.sequence

In [24]:
class EnsembleNode(object):
    
    def __init__(self, propagators, sequence=[], depth=0):
        self.propagators = propagators  # will be list of Qobj for ensemble
        self.sequence = sequence
        self.depth = depth
        
        self.children = {}
    
    def has_children(self):
        return len(self.children) > 0
    
    def evaluate(self, Utarget, pulses_ensemble, reward_dict, max_depth=6):
        """If the node isn't at max_depth, then create children and
        evaluate each individually. If the node is at max_depth, then
        calculate the reward and add the sequence/reward pair to
        reward_dict.
        
        Arguments:
            pulses: An array of unitary operators representing all actions
                that can be applied to the system.
        Returns: The maximum reward seen by the node or its children, and
            the corresponding sequence.
            
        """
        if self.depth < max_depth:
            max_reward = 0
            max_reward_sequence = []
            for i in range(len(pulses_ensemble[0])):
                propagators = []
                for j in range(len(pulses_ensemble)):
                    propagators.append(pulses_ensemble[j][i] * self.propagators[j])
                child = EnsembleNode(propagators,
                             self.sequence + [i],
                             depth=self.depth + 1)
                r, s = child.evaluate(Utarget, pulses_ensemble, reward_dict, max_depth)
                if r > max_reward:
                    max_reward = r
                    max_reward_sequence = s
            return max_reward, max_reward_sequence
        else:
            fidelities = [np.clip(
                qt.metrics.average_gate_fidelity(p, Utarget),
                0, 1
            ) for p in self.propagators]
            fidelity = np.nanmean(fidelities)
            reward = - np.log10(1.0 - fidelity + 1e-100)
            sequence_str = ','.join([str(a) for a in self.sequence])
            reward_dict[sequence_str] = reward
            return reward, self.sequence

## Identify primitives

In [173]:
# reward_dict = {}
    
# root = Node(Utarget)
# max_reward, max_reward_sequence = root.evaluate(
#     Utarget, pulses, reward_dict
# )

# print(f'Max reward was {max_reward}\nSequence:\t{max_reward_sequence}')

In [61]:
reward_dict = {}
    
root = EnsembleNode([Utarget] * ensemble_size)
max_reward, max_reward_sequence = root.evaluate(
    Utarget, pulses_ensemble, reward_dict
)

print(f'Max reward was {max_reward}\nSequence:\t{max_reward_sequence}')

Max reward was 6.134934258595179
Sequence:	[3, 4, 2, 4, 3, 1]


In [64]:
sequences = list(reward_dict.keys())

In [65]:
rewards = list(reward_dict.values())

In [66]:
sequences.sort(key=lambda s: -reward_dict[s])

In [51]:
# (pulses_ensemble[0][0] * pulses_ensemble[0][4] * pulses_ensemble[0][2] * 
#  pulses_ensemble[0][0] * pulses_ensemble[0][1] * pulses_ensemble[0][3])

In [63]:
reward_dict['1,1,4,1,1,4'], reward_dict['3,4,2,4,3,1'], 

(2.824067141604811, 6.134934258595179)

In [67]:
sequences[:33]

['3,4,2,4,3,1',
 '3,1,3,4,2,4',
 '4,2,4,3,1,3',
 '4,3,1,3,4,2',
 '1,3,4,2,4,3',
 '2,4,3,1,3,4',
 '1,3,1,2,4,2',
 '2,1,3,1,2,4',
 '3,1,2,4,2,1',
 '1,2,4,2,1,3',
 '2,4,2,1,3,1',
 '4,2,1,3,1,2',
 '1,4,3,2,3,4',
 '3,4,1,4,3,2',
 '4,3,2,3,4,1',
 '2,3,4,1,4,3',
 '3,2,3,4,1,4',
 '4,1,4,3,2,3',
 '1,2,3,2,1,4',
 '2,1,4,1,2,3',
 '2,3,2,1,4,1',
 '3,2,1,4,1,2',
 '4,1,2,3,2,1',
 '1,4,1,2,3,2',
 '1,2,2,4,3,1',
 '2,2,4,3,1,1',
 '2,4,3,1,1,2',
 '3,1,1,2,2,4',
 '1,1,2,2,4,3',
 '4,3,1,1,2,2',
 '4,4,3,3,1,2',
 '2,4,4,3,3,1',
 '3,1,2,4,4,3']

In [68]:
[reward_dict[s] for s in sequences[:24]]

[6.134934258595179,
 6.134934258463608,
 6.134934258463608,
 6.134934258463608,
 6.134934258332038,
 6.134934258332038,
 6.134934109657306,
 6.134934109657306,
 6.134934109657306,
 6.1349341095257355,
 6.1349341095257355,
 6.134934109394165,
 6.134655160980322,
 6.134655160980322,
 6.134655160980322,
 6.134655160848836,
 6.134655160848836,
 6.134655160848836,
 6.134654866254428,
 6.134654866254428,
 6.134654866254428,
 6.134654866254428,
 6.134654866254428,
 6.134654866122942]

## Construct primitives

In [194]:
primitives_ensemble = []  # list of list of Qobj

for j in range(len(pulses_ensemble)):
    primitives = []
    for i in range(24):
        propagator = qt.identity(pulses_ensemble[0][0].dims[0])
        sequence = sequences[i]
        for p in sequence.split(','):
            propagator = pulses_ensemble[j][int(p)] * propagator
        primitives.append(propagator)
    primitives_ensemble.append(primitives)

In [215]:
# qt.hinton(primitives[0] + Utarget)
# qt.hinton(primitives[1] + Utarget)

## Build longer pulse sequences from primitives

In [200]:
reward_dict_2 = {}

root = EnsembleNode([Utarget]*3)
max_reward, max_reward_sequence = root.evaluate(
    Utarget, primitives_ensemble, reward_dict_2, max_depth=2)

In [201]:
sequences_2 = list(reward_dict_2.keys())
rewards_2 = list(reward_dict_2.values())

In [212]:
sequences_2.sort(key=lambda s: -reward_dict_2[s])

In [213]:
sequences_2[:17]

['0,16',
 '1,19',
 '2,21',
 '3,22',
 '4,14',
 '5,15',
 '6,12',
 '7,13',
 '16,0',
 '19,1',
 '21,2',
 '22,3',
 '12,6',
 '13,7',
 '14,4',
 '15,5',
 '4,3']

In [208]:
rewards_2.sort(reverse=True)

In [211]:
rewards_2[:17]

[4.138787196443834,
 4.138787196443834,
 4.138787196443834,
 4.138787196443834,
 4.138787196443171,
 4.138787196443171,
 4.138787196443171,
 4.138787196443171,
 4.138787196443171,
 4.138787196443171,
 4.138787196443171,
 4.138787196443171,
 4.1387871964425065,
 4.1387871964425065,
 4.1387871964425065,
 4.1387871964425065,
 4.034173108172478]

In [214]:
for i in range(16):
    inds = sequences_2[i].split(',')
    print(sequences[int(inds[0])] + ',' + sequences[int(inds[1])])

0,3,3,0,3,3,0,1,1,0,1,1
1,0,0,1,0,0,1,2,2,1,2,2
2,1,1,2,1,1,2,3,3,2,3,3
3,2,2,3,2,2,3,0,0,3,0,0
0,0,1,0,0,1,2,2,1,2,2,1
1,1,2,1,1,2,3,3,2,3,3,2
2,2,3,2,2,3,0,0,3,0,0,3
3,3,0,3,3,0,1,1,0,1,1,0
0,1,1,0,1,1,0,3,3,0,3,3
1,2,2,1,2,2,1,0,0,1,0,0
2,3,3,2,3,3,2,1,1,2,1,1
3,0,0,3,0,0,3,2,2,3,2,2
0,0,3,0,0,3,2,2,3,2,2,3
1,1,0,1,1,0,3,3,0,3,3,0
2,2,1,2,2,1,0,0,1,0,0,1
3,3,2,3,3,2,1,1,2,1,1,2
